# Tugas 2 Normalisasi text

1. Punctuation process menghilangkan tanda baca dan simbol

2. Stopword

3. Tokenisasi

4. Steeming

5. Feature extraction dan membentuk VSM dalam term frequency, logarithm freqency, one-hot encoding,  TF-IDF

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=b00297395138f0e23ac5939c82d16b475635fdbcb81b785092d678b9eaede5a8
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [6]:
#import library
import pandas as pd
import numpy as np

In [7]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
#load data
df = pd.read_excel("/content/drive/MyDrive/ppw/tugas/data-tugas2/DF_PTA.xlsx")
df.head()

,NIM,Judul,Abstrak,Program Studi,Penulis,Dosen Pembimbing 1,Dosen Pembimbing 2,Label
0,4.041110e+10,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,sistem informasi akademik siakad merupaka...,Teknik Informatika,Penulis : A.Ubaidillah S.Kom,Dosen Pembimbing I : Budi Setyono M.T,Dosen Pembimbing II :Hermawan S.T,RPL
1,4.041110e+10,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,"Penulis : M. Basith Ardianto,","Dosen Pembimbing I : Drs. Budi Soesilo, MT","Dosen Pembimbing II :Koko Joni, ST",RPL
2,4.041110e+10,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,web server adalah sebuah perangkat lunak serve...,Teknik Informatika,"Penulis : Akhmad Suyandi, S.Kom","Dosen Pembimbing I : Drs. Budi Soesilo, M.T","Dosen Pembimbing II :Hermawan, ST, MT",RPL
3,7.041110e+10,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,penjadwalan kuliah di perguruan tinggi me...,Teknik Informatika,Penulis : Heri Supriyanto,"Dosen Pembimbing I : Mulaab, S.Si., M.Kom","Dosen Pembimbing II :Firli Irhamni, ST., M.Kom",komputasi
4,8.041110e+10,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,Penulis : Septian Rahman Hakim,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Haryanto, S.T., M.T.",komputasi


In [14]:
df.shape

(853, 8)

In [15]:
df = df.dropna()

## Menghilangkan Tanda Baca

In [16]:
def remove_tanda_baca(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['Abstrak'] = df['Abstrak'].apply(remove_tanda_baca)
df['Abstrak'].head(20)

0     sistem  informasi  akademik   siakad  merupaka...
1     berjalannya koneksi jaringan komputer dengan l...
2     web server adalah sebuah perangkat lunak serve...
3     penjadwalan  kuliah  di  perguruan  tinggi  me...
4     seiring perkembangan teknologi yang ada diduni...
5     gerak pekerja ada pada game yang memiliki genr...
6     perkembangan game yang semakin pesat  memberik...
7     sistem pengenalan wajah adalah suatu sistem un...
8     teknologi mobile game beroperating system open...
9     kantor badan kepegawaian kota bangkalan adalah...
10    penyusunan sistem informasi dinas perindustria...
11    perusahaan pemerintah maupun swasta mempunyai ...
12    pelayanan makanan bagi pasien rawat inap di ru...
13    penyusunan sistem pendukung keputusan pemiliha...
14    sidik jari adalah salah satu karakteristik fis...
15    di indonesia masalah perkembangan gizi adalah ...
16    pengenalan tulisan tangan merupakan topik pene...
17    citra senyum merupakan salah satu fitur bi

## Word Normalizaation

In [17]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['Abstrak'] = df['Abstrak'].apply(remove_numbers)
df['Abstrak']

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
823    investasi saham selama ini memiliki resiko ker...
824    information retrieval  ir  merupakan pengambil...
825    klasifikasi citra merupakan proses pengelompok...
826    identifikasi atribut pejalan kaki merupakan sa...
827    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 786, dtype: object

In [18]:
#proses casefolding
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['Abstrak'] = df['Abstrak'].apply(casefolding)
df['Abstrak']

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
823    investasi saham selama ini memiliki resiko ker...
824    information retrieval  ir  merupakan pengambil...
825    klasifikasi citra merupakan proses pengelompok...
826    identifikasi atribut pejalan kaki merupakan sa...
827    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 786, dtype: object

## Tokenisasi

In [19]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df['review_token'] = df['Abstrak'].apply(lambda sentence: nltk.word_tokenize(sentence))
df['review_token']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0      [sistem, informasi, akademik, siakad, merupaka...
1      [berjalannya, koneksi, jaringan, komputer, den...
2      [web, server, adalah, sebuah, perangkat, lunak...
3      [penjadwalan, kuliah, di, perguruan, tinggi, m...
4      [seiring, perkembangan, teknologi, yang, ada, ...
                             ...                        
823    [investasi, saham, selama, ini, memiliki, resi...
824    [information, retrieval, ir, merupakan, pengam...
825    [klasifikasi, citra, merupakan, proses, pengel...
826    [identifikasi, atribut, pejalan, kaki, merupak...
827    [topik, deteksi, objek, telah, menarik, perhat...
Name: review_token, Length: 786, dtype: object

In [20]:
df['review_token'].to_excel('normalisasi-data.xlsx', index=False)

In [21]:
#Normalisasi kata tidak baku
normalize = pd.read_excel("/content/drive/MyDrive/ppw/tugas/data-tugas2/normalisasi-data.xlsx")

normalize_word_dict = {}

for row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

0     [sistem, informasi, akademik, siakad, merupaka...
1     [berjalannya, koneksi, jaringan, komputer, den...
2     [web, server, adalah, sebuah, perangkat, lunak...
3     [penjadwalan, kuliah, di, perguruan, tinggi, m...
4     [seiring, perkembangan, teknologi, yang, ada, ...
5     [gerak, pekerja, ada, pada, game, yang, memili...
6     [perkembangan, game, yang, semakin, pesat, mem...
7     [sistem, pengenalan, wajah, adalah, suatu, sis...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, maupun, swasta, mempu...
12    [pelayanan, makanan, bagi, pasien, rawat, inap...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, adalah, salah, satu, karakterist...
15    [di, indonesia, masalah, perkembangan, gizi, a...
16    [pengenalan, tulisan, tangan, merupakan, topik...
17    [citra, senyum, merupakan, salah, satu, fi

## Stopwords Removal

In [22]:
#Stopword Removal
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal'].head(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0     [sistem, informasi, akademik, siakad, sistem, ...
1     [berjalannya, koneksi, jaringan, komputer, lan...
2     [web, server, perangkat, lunak, server, berfun...
3     [penjadwalan, kuliah, perguruan, kompleks, per...
4     [seiring, perkembangan, teknologi, didunia, mu...
5     [gerak, pekerja, game, memiliki, genre, rts, r...
6     [perkembangan, game, pesat, alternative, pemin...
7     [sistem, pengenalan, wajah, sistem, mengenali,...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, swasta, tujuan, dicap...
12    [pelayanan, makanan, pasien, rawat, inap, ruma...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, salah, karakteristik, fisik, man...
15    [indonesia, perkembangan, gizi, perhatian, sta...
16    [pengenalan, tulisan, tangan, topik, penelitia...
17    [citra, senyum, salah, fitur, biometrik, d

In [23]:
df['stopwords_removal'].to_excel('stopwords-removal.xlsx', index=False)

In [24]:
#stopword removal 2
data_stopwords = pd.read_excel("/content/drive/MyDrive/ppw/tugas/data-tugas2/stopwords-removal.xlsx")
print(data_stopwords)

def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final'].head(20)

                                     stopwords_removal
0    ['sistem', 'informasi', 'akademik', 'siakad', ...
1    ['berjalannya', 'koneksi', 'jaringan', 'komput...
2    ['web', 'server', 'perangkat', 'lunak', 'serve...
3    ['penjadwalan', 'kuliah', 'perguruan', 'komple...
4    ['seiring', 'perkembangan', 'teknologi', 'didu...
..                                                 ...
823  ['investasi', 'saham', 'memiliki', 'resiko', '...
824  ['information', 'retrieval', 'ir', 'pengambila...
825  ['klasifikasi', 'citra', 'proses', 'pengelompo...
826  ['identifikasi', 'atribut', 'pejalan', 'kaki',...
827  ['topik', 'deteksi', 'objek', 'menarik', 'perh...

[828 rows x 1 columns]


0     [sistem, informasi, akademik, siakad, sistem, ...
1     [berjalannya, koneksi, jaringan, komputer, lan...
2     [web, server, perangkat, lunak, server, berfun...
3     [penjadwalan, kuliah, perguruan, kompleks, per...
4     [seiring, perkembangan, teknologi, didunia, mu...
5     [gerak, pekerja, game, memiliki, genre, rts, r...
6     [perkembangan, game, pesat, alternative, pemin...
7     [sistem, pengenalan, wajah, sistem, mengenali,...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, swasta, tujuan, dicap...
12    [pelayanan, makanan, pasien, rawat, inap, ruma...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, salah, karakteristik, fisik, man...
15    [indonesia, perkembangan, gizi, perhatian, sta...
16    [pengenalan, tulisan, tangan, topik, penelitia...
17    [citra, senyum, salah, fitur, biometrik, d

## STEMMING

In [25]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}
for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''

In [26]:
print(len(term_dict))
print("-----------------------------")

8053
-----------------------------


In [27]:
for term in term_dict:
  term_dict[term] = stemming(term)
  print(term,":",term_dict[term])

print(term_dict)
print("-----------------------------")

Output streaming akan dipotong hingga 5000 baris terakhir.
pencariannya : cari
prototype : prototype
mendeskripsikan : deskripsi
subjek : subjek
predikat : predikat
sparql : sparql
arq : arq
jena : jena
domain : domain
detikinet : detikinet
com : com
standarisasi : standar
wc : wc
optimalisasi : optimalisasi
precission : precission
smkn : smkn
buduran : budur
sidoarjo : sidoarjo
jasa : jasa
boga : boga
busana : busana
butik : butik
akomodasi : akomodasi
perhotelan : hotel
kecantikan : cantik
bakat : bakat
minatnya : minat
additive : additive
saw : saw
komplekdalam : komplekdalam
goal : goal
atributes : atributes
terbobot : bobot
alternatifpada : alternatifpada
diagonal : diagonal
diapca : diapca
alasan : alas
mengubah : ubah
memadukan : madu
berguna : guna
diakui : aku
terurut : urut
short : short
butuhkan : butuh
pengkodingan : pengkodingan
universal : universal
sejenis : jenis
ponsel : ponsel
bit : bit
chipertext : chipertext
advanced : advanced
erosi : erosi
kejadian : jadi
dibumi :

In [28]:
def get_stemming(document):
  return [term_dict[term] for term in document]

In [29]:
df['stemming'] = df['stopwords_removal_final'].swifter.apply(get_stemming)

Pandas Apply:   0%|          | 0/786 [00:00<?, ?it/s]

In [30]:
print(df['stemming'])

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, fungsi...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
823    [investasi, saham, milik, resiko, rugi, gera, ...
824    [information, retrieval, ir, ambil, informasi,...
825    [klasifikasi, citra, proses, kelompok, piksel,...
826    [identifikasi, atribut, pejal, kaki, salah, te...
827    [topik, deteksi, objek, tarik, perhati, kemban...
Name: stemming, Length: 786, dtype: object


In [31]:
df.head(20)

,NIM,Judul,Abstrak,Program Studi,Penulis,Dosen Pembimbing 1,Dosen Pembimbing 2,Label,review_token,comment_normalize,stopwords_removal,stopwords_removal_final,stemming
0,4.041110e+10,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,sistem informasi akademik siakad merupaka...,Teknik Informatika,Penulis : A.Ubaidillah S.Kom,Dosen Pembimbing I : Budi Setyono M.T,Dosen Pembimbing II :Hermawan S.T,RPL,"[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, merupaka...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ...","[sistem, informasi, akademik, siakad, sistem, ..."
1,4.041110e+10,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,"Penulis : M. Basith Ardianto,","Dosen Pembimbing I : Drs. Budi Soesilo, MT","Dosen Pembimbing II :Koko Joni, ST",RPL,"[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, den...","[berjalannya, koneksi, jaringan, komputer, lan...","[berjalannya, koneksi, jaringan, komputer, lan...","[jalan, koneksi, jaring, komputer, lancar, gan..."
2,4.041110e+10,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,web server adalah sebuah perangkat lunak serve...,Teknik Informatika,"Penulis : Akhmad Suyandi, S.Kom","Dosen Pembimbing I : Drs. Budi Soesilo, M.T","Dosen Pembimbing II :Hermawan, ST, MT",RPL,"[web, server, adalah, sebuah, perangkat, lunak...","[web, server, adalah, sebuah, perangkat, lunak...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, berfun...","[web, server, perangkat, lunak, server, fungsi..."
3,7.041110e+10,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,penjadwalan kuliah di perguruan tinggi me...,Teknik Informatika,Penulis : Heri Supriyanto,"Dosen Pembimbing I : Mulaab, S.Si., M.Kom","Dosen Pembimbing II :Firli Irhamni, ST., M.Kom",komputasi,"[penjadwalan, kuliah, di, perguruan, tinggi, m...","[penjadwalan, kuliah, di, perguruan, tinggi, m...","[penjadwalan, kuliah, perguruan, kompleks, per...","[penjadwalan, kuliah, perguruan, kompleks, per...","[jadwal, kuliah, guru, kompleks, masalah, vari..."
4,8.041110e+10,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,Penulis : Septian Rahman Hakim,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Haryanto, S.T., M.T.",komputasi,"[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, yang, ada, ...","[seiring, perkembangan, teknologi, didunia, mu...","[seiring, perkembangan, teknologi, didunia, mu...","[iring, kembang, teknologi, dunia, muncul, tek..."
5,7.041110e+10,Gerak Pekerja Pada Game Real Time Strategy Men...,gerak pekerja ada pada game yang memiliki genr...,Teknik Informatika,Penulis : Adi Chandra Laksono,"Dosen Pembimbing I : Kurniawan Eka P, S.Kom., Msc","Dosen Pembimbing II :Arik Kurniawati, S.Kom., ...",komputasi,"[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, ada, pada, game, yang, memili...","[gerak, pekerja, game, memiliki, genre, rts, r...","[gerak, pekerja, game, memiliki, genre, rts, r...","[gerak, kerja, game, milik, genre, rts, real, ..."
6,7.041110e+10,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,perkembangan game yang semakin pesat memberik...,Teknik Informatika,Penulis : NURRACHMAT,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Kurniawan Eka Permana, S....",komputasi,"[perkembangan, game, yang, semakin, pesat, mem...","[perkembangan, game, yang, semakin, pesat, mem...","[perkembangan, game, pesat, alternative, pemin...","[perkembangan, game, pesat, alternative, pemin...","[kembang, game, pesat, alternative, minat, ben..."
7,7.041110e+10,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,sistem pengenalan wajah adalah suatu sistem un...,Teknik Informatika,Penulis : Muhammad Choirur Rozi,"Dosen Pembimbing I : Dr. Ar

In [32]:
df.to_excel('df.xlsx', index=False)

## Feature extraction dan membentuk VSM dalam term frequency

mengambil dokumen atau teks, melakukan ekstraksi fitur dengan menghitung frekuensi kemunculan kata-kata (TF) dalam dokumen, dan kemudian merepresentasikan dokumen-dokumen tersebut dalam bentuk vektor dalam ruang VSM

In [33]:
# mengambil data hasil proses pre-prosessing sebelumnya untuk kategori dan text hasil stemming
datap = pd.read_excel("df.xlsx", usecols=["stemming"])
datap.columns = ["Abstrak"]
datap

,Abstrak
0,"['sistem', 'informasi', 'akademik', 'siakad', ..."
1,"['jalan', 'koneksi', 'jaring', 'komputer', 'la..."
2,"['web', 'server', 'perangkat', 'lunak', 'serve..."
3,"['jadwal', 'kuliah', 'guru', 'kompleks', 'masa..."
4,"['iring', 'kembang', 'teknologi', 'dunia', 'mu..."
...,...
781,"['investasi', 'saham', 'milik', 'resiko', 'rug..."
782,"['information', 'retrieval', 'ir', 'ambil', 'i..."
783,"['klasifikasi', 'citra', 'proses', 'kelompok',..."
784,"['identifikasi', 'atribut', 'pejal', 'kaki', '..."


In [34]:
datap.to_excel('datap.xlsx', index=False)

In [35]:
datap.shape

(786, 1)

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Baca data dari DataFrame
data = datap["Abstrak"]

# Inisialisasi CountVectorizer
cv = CountVectorizer()

# Transformasikan data menggunakan CountVectorizer
term_frequency_matrix = cv.fit_transform(data)

# Dapatkan daftar kata-kata (fitur)
feature_names = cv.get_feature_names_out()

# Buat DataFrame untuk term frequency
term_frequency_df = pd.DataFrame(term_frequency_matrix.toarray(), columns=feature_names)

# Tampilkan DataFrame term frequency
print(term_frequency_df)


     aalysis  aam  abad  abadi  abai  abdi  ability  abjad  absensi  absolut  \
0          0    0     0      0     0     0        0      0        0        0   
1          0    0     0      0     0     0        0      0        0        0   
2          0    0     0      0     0     0        0      0        0        0   
3          0    0     0      0     0     0        0      0        0        0   
4          0    0     0      0     0     0        0      0        0        0   
..       ...  ...   ...    ...   ...   ...      ...    ...      ...      ...   
781        0    0     0      0     0     0        0      0        0        0   
782        0    0     0      0     0     0        0      0        0        0   
783        0    0     0      0     0     0        0      0        0        0   
784        0    0     0      0     0     0        0      0        0        0   
785        0    0     0      0     0     0        0      0        0        0   

     ...  zara  zat  zcz  zf  zona  zon

In [37]:
term_frequency_df.to_excel('term_frequency_df.xlsx', index=False)

## Logarithm frequency

Logarithm frequency adalah sebuah metode transformasi yang digunakan dalam analisis data untuk mengubah data frekuensi menjadi skala logaritma

$If_{i, j}=\left\{\begin{array}{ll}
In(tf_{i,j} + 1 ), & \text { if } t f_{i, j}>0 \\
0, & \text { otherwise }
\end{array}\right\}.$

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
log_vectorizer = TfidfVectorizer(use_idf=False, smooth_idf=False, norm=None)
X_log = log_vectorizer.fit_transform(datap['Abstrak'].tolist())

log_terms = log_vectorizer.get_feature_names_out()
df_log = pd.DataFrame(data = X_log.toarray(),columns = log_terms)
df_log

,aalysis,aam,abad,abadi,abai,abdi,ability,abjad,absensi,absolut,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_log.to_excel('df_log.xlsx', index=False)

## Binary Frequency

One-hot encoding digunakan untuk mengubah data kategori atau kualitatif menjadi representasi biner yang dapat digunakan dalam perhitungan dan analisis statistik.


$n_{i, j}=\left\{\begin{array}{ll}
1, & \text { if } t f_{i, j}>0 \\
0, & \text { otherwise }
\end{array}\right\}.$

In [56]:
df_encoded = pd.get_dummies(df_log)


In [57]:
df_encoded

,aalysis,aam,abad,abadi,abai,abdi,ability,abjad,absensi,absolut,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kode di atas menggunakan pd.get_dummies untuk menghasilkan one-hot encoding dari kolom "Abstrak" dalam DataFrame datap. Setiap kategori atau nilai unik dalam kolom tersebut akan diubah menjadi kolom tersendiri dengan nilai 1 atau 0 untuk setiap baris, sesuai dengan keberadaan kategori tersebut dalam baris tersebut

In [49]:
# Simpan DataFrame ke dalam file Excel
one_hot_df = df_log  # Assign DataFrame yang telah diubah ke one_hot_df
one_hot_df.to_excel('one_hot_df.xlsx', index=False)

## TF - IDF

TF-IDF adalah singkatan dari "Term Frequency-Inverse Document Frequency" dalam pemrosesan bahasa alami dan analisis teks. Ini adalah metode statistik yang digunakan untuk mengukur pentingnya sebuah kata atau istilah dalam suatu dokumen terhadap koleksi dokumen yang lebih besar.

In [50]:
# join list of token as single document string
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
datap["text_join"] = datap["Abstrak"].apply(join_text_list)

datap["text_join"].head()

0    sistem informasi akademik siakad sistem inform...
1    jalan koneksi jaring komputer lancar ganggu ha...
2    web server perangkat lunak server fungsi terim...
3    jadwal kuliah guru kompleks masalah variabel t...
4    iring kembang teknologi dunia muncul teknologi...
Name: text_join, dtype: object

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

# banyaknya term yang akan digunakan,
# di pilih berdasarkan top max_features
# yang diurutkan berdasarkan term frequency seluruh corpus
max_features = 100000

# Feature Engineering
print ("------- TF-IDF data -------")

tf_idf = TfidfVectorizer(max_features=max_features, binary=True)
tfidf_mat = tf_idf.fit_transform(datap["text_join"]).toarray()

print("TF-IDF ", type(tfidf_mat), tfidf_mat.shape)

------- TF-IDF data -------
TF-IDF  <class 'numpy.ndarray'> (786, 5948)


In [52]:
terms = tf_idf.get_feature_names_out()

# sum tfidf frequency of each term through documents
sums = tfidf_mat.sum(axis=0)

# connecting term to its sums frequency
datas = []
for col, term in enumerate(terms):
    datas.append((term, sums[col] ))

ranking = pd.DataFrame(datas, columns=['term','rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
1921,hasil,24.052973
4959,sistem,22.790998
3294,metode,22.783679
5364,teliti,22.307769
3532,nilai,20.856780
...,...,...
3509,nerve,0.147967
3624,onh,0.147967
3092,marking,0.147967
392,automasi,0.147967


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize

max_features = 10000

# calc TF vector
cvect = CountVectorizer(max_features=max_features)
TF_vector = cvect.fit_transform(datap["text_join"])

# normalize TF vector
normalized_TF_vector = normalize(TF_vector, norm='l1', axis=1)

# calc IDF
tfidf = TfidfVectorizer(max_features=max_features, smooth_idf=False)
tfs = tfidf.fit_transform(datap["text_join"])
IDF_vector = tfidf.idf_

# hitung TF x IDF sehingga dihasilkan TFIDF matrix / vector
tfidf_mat = normalized_TF_vector.multiply(IDF_vector).toarray()

In [54]:
# hasil dari tf-idf
dfbtf= pd.DataFrame(data=tfidf_mat,columns=[terms])
dfbtf

,aalysis,aam,abad,abadi,abai,abdi,ability,abjad,absensi,absolut,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
dfbtf.to_excel('dfbtf.xlsx', index=False)